In [1]:
import json, os
import numpy as np

# 選擇資料夾
folder_name = 'json_public' # json_public json_random

# root/folder/subfolder/flie.json
root = os.getcwd()
folder_path = os.path.join(root, folder_name)

print(os.listdir(folder_path))

subfolder_names = [
    'GCN_cora',
    'GCN_v2_cora',
    
    'GCN_citeseer',
    'GCN_v2_citeseer',
    
    'GCN_pubmed',
    'GCN_v2_pubmed',
]

# 讀取所有 json 檔案
dict_dict_json = {}  # {key: file_name, value: json_file_date}

for subfolder_name in subfolder_names:
    subfolder_path = os.path.join(folder_path, subfolder_name)
    #subfolder_path = folder_path
    print(subfolder_path)
    for file_name in os.listdir(subfolder_path):
        #print(f"\t{file_name}")
        file_path = os.path.join(subfolder_path, file_name)
        with open(file_path, 'r') as f:
            dict_dict_json[file_name] = json.loads(f.read())

['GCN_citeseer', 'GCN_cora', 'GCN_pubmed', 'GCN_v2_citeseer', 'GCN_v2_cora', 'GCN_v2_pubmed']
C:\Users\BingYu\Documents\GitHub\HGNN-Node-Classification\json_random\GCN_cora
C:\Users\BingYu\Documents\GitHub\HGNN-Node-Classification\json_random\GCN_v2_cora
C:\Users\BingYu\Documents\GitHub\HGNN-Node-Classification\json_random\GCN_citeseer
C:\Users\BingYu\Documents\GitHub\HGNN-Node-Classification\json_random\GCN_v2_citeseer
C:\Users\BingYu\Documents\GitHub\HGNN-Node-Classification\json_random\GCN_pubmed
C:\Users\BingYu\Documents\GitHub\HGNN-Node-Classification\json_random\GCN_v2_pubmed


In [2]:
import openpyxl
xlsx_name = f"info_{folder_name}.xlsx"
xlsx_path = os.path.join(root, xlsx_name)

if os.path.isfile(xlsx_path):
    wb = openpyxl.load_workbook(xlsx_path)
    print(f"path: {xlsx_path} 已存在.")
else:
    wb = openpyxl.Workbook()
    print(f"path: {xlsx_path}")

path: C:\Users\BingYu\Documents\GitHub\HGNN-Node-Classification\info_json_random.xlsx


In [3]:
# [json_file_date, ...]
list_dict_json = list(dict_dict_json.values())

dict_json_keys = list(list_dict_json[0].keys())

file_keys = ['project_name', 'save_time']

same_keys = ['dataset', 'num_class', 'validation_split', 'loss', 'optimizers', 'learning_rate', 'epochs', 'batch_size',
            'model_name', 'layer1_list', 'layer2_list', 'fc_list', 'dropout_rate']

#    GCN : 'image_adjs'
# Cb GCN : 'image_adj', 'num_K'
#   PGCN : 'polar_adjs', 'input_shape', 'num_theta', 'num_radius'
# CbPGCN : 'polar_adj', 'num_K', 'input_shape', 'num_theta', 'num_radius'
#    CNN : 'kernel_size'
main_keys = ['image_adjs', 'image_adj', 'polar_adjs', 'polar_adj', 'kernel_size']
submain_keys = ['num_K', 'num_theta', 'num_radius']

date_keys = ['train_acc', 'train_loss', 'val_acc', 'val_loss', 'test_acc', 'test_loss', 'train_time', 'total_params']
subdata_keys = ['gpu_name', 'test_time']
history_keys = ['history_epoch', 'history_loss', 'history_accuracy', 'history_val_loss', 'history_val_accuracy']

# 未使用
None_keys = ['is_features', 'isBatchNorm', 'use_residual', 'use_inception']

all_keys = file_keys+same_keys+main_keys+submain_keys+date_keys+subdata_keys+history_keys+None_keys

other_keys = []
for key in dict_json_keys:
    if key not in all_keys:
        other_keys.append(key)
print(f'other_keys = {other_keys}')

not_keys = []
for key in all_keys:
    if key not in dict_json_keys:
        not_keys.append(key)
print(f'not_keys = {not_keys}')

other_keys = ['weight_decay', 'model', 'GPU']
not_keys = ['project_name', 'num_class', 'validation_split', 'loss', 'batch_size', 'model_name', 'layer1_list', 'layer2_list', 'fc_list', 'dropout_rate', 'image_adjs', 'image_adj', 'polar_adjs', 'polar_adj', 'kernel_size', 'num_K', 'num_theta', 'num_radius', 'gpu_name', 'is_features', 'isBatchNorm', 'use_residual', 'use_inception']


In [4]:
info_keys = file_keys+same_keys+main_keys+submain_keys+date_keys+subdata_keys+None_keys+other_keys+history_keys

if 'Main' in wb.sheetnames:
    ws = wb['Main']
else:
    ws = wb.create_sheet('Main')
    ws.append(info_keys)


for file_name, dict_value in dict_dict_json.items():
    row_data = []
    for key in info_keys:
        
        if key not in dict_value.keys():
            data = '-'
        else:
            data = dict_value[key]
        
        if type(data) == list:
            data = str(data)
        elif data == None:
            data = 'None'
        
        row_data.append(data)
    ws.append(row_data)

In [5]:
data_keys = [
    'train_acc', 'train_loss', 'val_acc', 'val_loss', 'test_acc', 'test_loss',
    'train_time', 'total_params'
]

table_dict = {}
for file_name, dict_value in dict_dict_json.items():

    table_key = f"{dict_value['model']}_{dict_value['dataset']}"

    # 增加一筆 table_key 欄位
    if table_key not in table_dict.keys():
        table_dict[table_key] = {}
        for data_key in data_keys:
            table_dict[table_key][data_key] = []
    # 添加資料
    for data_key in data_keys:
        table_dict[table_key][data_key].append(dict_value[data_key])

if 'Select' in wb.sheetnames:
    ws = wb['Select']
else:
    ws = wb.create_sheet('Select')
    ws.append(['table_key'] + data_keys)

for table_key in subfolder_names:
    if table_key in table_dict.keys():
        item_dict = table_dict[table_key]
    else:
        print(f'table_key = {table_key} continue')
        continue
    ####<  >####
    row_data = [table_key]
    for data_key in data_keys:
        data = np.array(item_dict[data_key])

        #if data_key in ['train_acc', 'val_acc', 'test_acc']:
        #    data = np.clip(data, 0, 1)
        #    if data.max() > 1:
        #        print(data)

        data_mean = np.mean(data)
        data_std = np.std(data)
        if data_key in ['train_acc', 'val_acc', 'test_acc']:
            data_str = f"{data_mean*100:5.2f}±{data_std*100:4.2f}"
        elif data_key in ['train_loss', 'val_loss', 'test_loss']:
            data_str = f"{data_mean:5.3f}±{data_std:5.3f}"
        elif data_key == 'train_time':
            data_str = f"{data_mean:6.1f}±{data_std:3.1f}"
        elif data_key == 'total_params':
            data_str = int(data_mean)
        row_data.append(data_str)
    ws.append(row_data)

In [6]:
if 'Sheet' in wb.sheetnames: wb.remove(wb['Sheet'])
wb.save(xlsx_path)